# El problema de trabajar con Valores Faltantes mediante Modelos

## Librerias

In [27]:
import janitor
import matplotlib.pyplot as plt 
import numpy as np  
import pandas as pd 
import scipy.stats  # trabjar con estadisticos
import seaborn as sns
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statsmodels.datasets

## Configuracion de las graficas

In [28]:
%matplotlib inline

# tamaño de los graficos
sns.set(
    rc={
        'figure.figsize':(8, 6)
    }
)

# estilo de los graficos
sns.set_style('whitegrid')

## Cargar los Datos

Cargamos un dataset de prueba sobre la calidad del aire proporcionado por la libreria statsmodels

In [29]:
airquality_df = (
    sm.datasets.get_rdataset('airquality')      # obtener datos
    .data                                       # acceder a los datos
    .clean_names(                               # cambiar formato de nombre de las variables
        case_type='snake'
    )
    .add_column('year',1973)                    # agregar nueva columna
    .assign(                                    # nueva columna con fecha
        date = lambda df: pd.to_datetime(df[['year', 'month', 'day']])
    )
    .sort_values(by='date')                     # ordenar por fecha
    .set_index('date')                          # indice fecha
)

airquality_df

,ozone,solar_r,wind,temp,month,day,year
date,,,,,,,
1973-05-01,41.0,190.0,7.4,67,5,1,1973
1973-05-02,36.0,118.0,8.0,72,5,2,1973
1973-05-03,12.0,149.0,12.6,74,5,3,1973
1973-05-04,18.0,313.0,11.5,62,5,4,1973
1973-05-05,NaN,NaN,14.3,56,5,5,1973
...,...,...,...,...,...,...,...
1973-09-26,30.0,193.0,6.9,70,9,26,1973
1973-09-27,NaN,145.0,13.2,77,9,27,1973
1973-09-28,14.0,191.0,14.3,75,9,28,1973


## Variables con Valores Faltantes

Verificamos las variables que contienen valores faltantes y hacemos un conteo de los mismos

In [30]:
(
    airquality_df
    .isna()
    .sum()
)

ozone      37
solar_r     7
wind        0
temp        0
month       0
day         0
year        0
dtype: int64

Vemos que solo dos variables poseen valores faltantes. Ozone posee 37 y solar_r posee 7.

## Modelo de Regresion Lineal sobre las variables con valores faltantes

Vamos a ajustar un modelo de regresion lineal (OLS) con statmodels sobre las variables temp y ozone que contiene valores faltantes.

In [32]:
(
    smf.ols(
        formula='temp ~ ozone',
        data= airquality_df
    )
    .fit()
    .summary()
    .tables[0]
)

Dep. Variable:,temp,R-squared:,0.488
Model:,OLS,Adj. R-squared:,0.483
Method:,Least Squares,F-statistic:,108.5
Date:,"Sat, 29 Jul 2023",Prob (F-statistic):,2.93e-18
Time:,07:24:02,Log-Likelihood:,-386.27
No. Observations:,116,AIC:,776.5
Df Residuals:,114,BIC:,782.1
Df Model:,1,,
Covariance Type:,nonrobust,,


El modelo OLS se aplica sobre 116 observaciones de las 157 existentes lo que indica que no tomo en cuenta los datos de ozone con valores faltantes. El R-squared es un indicador del ajuste del modelo sobre los datos. Se obtuvo un valor de 0.488